## Matting
Using %%timeit obtained 419 ms ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [1]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:

def mix_images(img_list, bg_path, colors, img_dest = None, sil_dest = None, show=False):
    bg = cv2.imread(bg_path)
    silhouettes = np.zeros(bg.shape, np.float64) 
    index = 0
    for paths in img_list:
        # Reading image, matte, and silhouette for each image
        img = cv2.imread(paths["image"]).astype(float)
        matte = cv2.imread(paths["matte"]).astype(float)/255
        sil = cv2.imread(paths["silhouette"]).astype(float)/255

        # Mix silhouettes, all silhouettes will be saved in the silhouettes variable
        silhouettes = cv2.multiply(1.0-sil,silhouettes)
        silhouettes = cv2.add(sil*colors[index], silhouettes)
        
        # Mix images and save them in the bg variable
        bg = bg.astype(float)
        fg = cv2.multiply(img,matte)
        
        # Multiply the background with ( 1 - alpha )
        back = cv2.multiply(1.0 - matte, bg)

        # Add the masked foreground and background.
        bg = cv2.add(fg, back)

        index+=1

    # Show results
    if(show):
        plt.imshow((bg/255)[:,:,::-1])
        plt.show()
        plt.imshow(silhouettes[:,:,::-1])
        plt.show()
    # Save images
    if img_dest != None:
        cv2.imwrite(img_dest,bg)
    if sil_dest != None:
        cv2.imwrite(sil_dest,silhouettes)

## Example of use mix_images()
```python
img_list = [
    {"image":"/home/josmar/proyectos/codes/datasets/ucb_gait_frames/images/018-060.jpg",
    "matte": "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/mattes/018-060.png",
    "silhouette":"/home/josmar/proyectos/codes/datasets/ucb_gait_frames/silhouettes/018-060.png"},
    {"image":"/home/josmar/proyectos/codes/datasets/ucb_gait_frames/images/002-008.jpg",
    "matte": "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/mattes/002-008.png",
    "silhouette":"/home/josmar/proyectos/codes/datasets/ucb_gait_frames/silhouettes/002-008.png"}
]
bg_path = "/media/josmar/Nuevo vol/Experiments/segmentation_codes/background.jpg"
colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255),(255, 0, 255)]
img_dest = "mix_img.jpg"
sil_dest = "mix_silhouettes.png"
mix_images(img_list, bg_path, colors, img_dest, sil_dest, show=True)
```

## Checking bg sizes

In [3]:
bg_dir = "/home/josmar/proyectos/codes/01_annotation_tools/test_data/backgrounds"
backgrounds = os.listdir(bg_dir)

for bg in backgrounds:
    img_path = os.path.join(bg_dir,bg)
    img = cv2.imread(img_path)
    if img.shape[0] != 720 and img.shape[1] != 1280:
        print(img_path)

## Aplying mix images to a folder

In [4]:
def get_iou(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [5]:
def augment_mix_images(n,img_dir,bg_dir,coco):
    count = 0
    backgrounds = os.listdir(bg_dir)
    images = os.listdir(os.path.join(img_dir, "images"))
    images = [image.split(".")[0] for image in images]

    category = "person"
    catIds = coco.getCatIds(catNms=[category])
    imgIds = coco.getImgIds(catIds=catIds )
    
    while count<n:
        imID = random.choices(imgIds, k=2)
        rand_imgs = coco.loadImgs(imID)
        bboxes = []
        image_list = []
        name_list = []
        for rand_img in rand_imgs:
            annIds = coco.getAnnIds(imgIds=rand_img['id'], catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)[0]

            bbox = anns["bbox"][:]
            bbox[2], bbox[3] = bbox[0]+bbox[2], bbox[1]+bbox[3] 
            bboxes.append(bbox)

            img_name = rand_img["file_name"].split(".")[0]

            paths = {"image":os.path.join(img_dir, "images", "{}.jpg".format(img_name)),
                    "matte": os.path.join(img_dir, "mattes", "{}.png".format(img_name)),
                    "silhouette":os.path.join(img_dir, "silhouettes", "{}.png".format(img_name))}
            image_list.append(paths)
            name_list.append(img_name)
        
        bg_name_ext = random.choice(backgrounds)
        bg_name = bg_name_ext.split(".")[0]
        bg_path = os.path.join(bg_dir, bg_name_ext)
        
        
        dest_name = "{}_{}_{}".format(bg_name ,name_list[0], name_list[1])
        
        img_dest = os.path.join(results_dir, "images", "{}.jpg".format(dest_name))
        sil_dest = os.path.join(results_dir, "silhouettes", "{}.png".format(dest_name))
        if (get_iou(bboxes[0],bboxes[1]) < 0.1 and (name_list[0].split("-")[0] != name_list[1].split("-")[0])):
            mix_images(image_list, bg_path, colors, img_dest, sil_dest)
            count+=1

            sys.stdout.write('\r'+  "Done: {}/{}".format(count,n))
            sys.stdout.flush()


In [6]:
# List bakcground images
bg_dir = "/home/josmar/proyectos/codes/01_annotation_tools/test_data/backgrounds"
backgrounds = os.listdir(bg_dir)

# List image files. Consider that images, silhouettes and mattes have the same notation ant the only thing that changes is the extension
# Example:
#           Image: 002-008.jpg
#           Matte: 002-008.png
#           Silhouette: 002-008.png
img_dir = "/home/josmar/proyectos/codes/datasets/ucb_gait_frames"
images = os.listdir(os.path.join(img_dir, "images"))
images = [image.split(".")[0] for image in images]

print("Total images: ", len(images))
print("Total backgrounds: ", len(backgrounds))



Total images:  12300
Total backgrounds:  33


In [7]:
from pycocotools.coco import COCO

annFile = "/home/josmar/proyectos/codes/datasets/ucb_gait_frames/annotations/filtered_ucb_gait_poly.json"
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=6.31s)
creating index...
index created!


In [8]:
n = 10000
results_dir = "/home/josmar/proyectos/codes/01_annotation_tools/test_data"
colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255),(255, 0, 255)]

augment_mix_images(n,img_dir,bg_dir,coco)

Done: 10000/10000